In [11]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import Genres as gen
import Artist as art
import Top5 as top

# Interactive Music Data Dashboard

This Python program provides an interactive interface to explore music data from an SQLite database. Users can check if the database exists, then fetch data on music genres, artists, and top 5 artists over selected years.

### Key Features:
1. **Database Check**: Users input the database name, and the program verifies its existence in the directory. If found, options for genre, artist, and top 5 data become available.

2. **Genre Data**: Fetch genre data for a specific year (1998–2020) and view it in a table or graph.

3. **Artist Data**: Search for artist-specific data and display it in a table or graph.

4. **Top 5 Artists**: Select a year range (1998–2020) to view the top 5 artists, with options to visualize the data.
Navigation: Use a "Back" button to return to the main menu.

In [ ]:
style = {"description width": "initial"}
db_widget = widgets.Text(description="Database: ",
                         value="CWDatabase.db",
                         placeholder="Enter database filename",
                         style=style)
db_check_widget = widgets.Button(description="Check Database")
back_button = widgets.Button(description="Back")

genre_widget = widgets.Button(description="Genre")
artist_widget = widgets.Button(description="Artist")
top5_widget = widgets.Button(description="Top 5")

lbl = widgets.Label()

genre_widget.disabled = True
artist_widget.disabled = True
top5_widget.disabled = True


def clicked_check_db(btn):
    """
    Checks the directory for the selected database. Raises a 'FileNotFound'
    error if the file is not present.
    """
    db = db_widget.value
    try:
        with open(db, "r"):
            lbl.value = f"Database '{db}' exists in directory."

            genre_widget.disabled = False
            artist_widget.disabled = False
            top5_widget.disabled = False

        display(genre_widget, artist_widget, top5_widget)
    except FileNotFoundError:
        lbl.value = f"Database '{db}' does not exist in directory."


def go_to_main(btn=None):
    """
    Back button to return to the main menu.
    """
    clear_output()
    display(db_widget, db_check_widget)


def fetch_table(fetch_func, *table_args):
    """
    Uses the selected fetch function while passing through the necessary
    arguments to create and display a formatted table.
    """
    clear_output()
    fetch_func(*table_args)


def fetch_graph(fetch_func, *graph_args):
    """
    Uses the selected fetch function while passing through the necessary
    arguments to create and display a formatted graph.
    """
    clear_output()
    fetch_func(*graph_args)


def clicked_genre(btn):
    clear_output()
    year_widget = widgets.IntText(
        description="Year: ",
        value=1998,
        min=1998, max=2020,
        placeholder="1999",
        style=style)
    fetch_button = widgets.Button(description="Fetch Genre Data", style=style)

    display(year_widget, fetch_button, back_button)

    def fetch_data(btn):
        """
        Fetches genre data once the 'fetch' button is pressed.
        """
        year = year_widget.value
        db = db_widget.value

        try:
            if not isinstance(year, int):
                clear_output()
                lbl.value = f"Year must be an integer."
                display(lbl, back_button)
                return

            if year < 1998 or year > 2020:
                clear_output()
                lbl.value = f"{year} is out of the date range 1998-2020."
                display(lbl, back_button)
                return

            df = gen.fetch_genre_data(db, year)
            if df.empty:
                clear_output()
                lbl.value = f"No data available for {year}."
                display(lbl, back_button)
                return

            clear_output()
            lbl.value = f"Data for {year} is available."
            table_widget = widgets.Button(
                description="View Table", style=style)
            graph_widget = widgets.Button(
                description="View Graph", style=style)
            display(lbl, table_widget, graph_widget, back_button)

            def table_wrap(btn):
                fetch_table(gen.format_table, df, year)
                display(graph_widget, back_button)

            def graph_wrap(btn):
                fetch_graph(gen.format_graph, df, year)
                display(table_widget, back_button)

        except Exception as e:
            lbl.value = f"Error: {e}"
            display(lbl, back_button)

        table_widget.on_click(table_wrap)
        graph_widget.on_click(graph_wrap)
        back_button.on_click(go_to_main)

    fetch_button.on_click(fetch_data)
    back_button.on_click(go_to_main)


def clicked_artist(btn):
    clear_output()

    artist_box = widgets.Text(description="Artist Name: ", style=style)
    fetch_artist_button = widgets.Button(
        description="Fetch Artist Data", style=style)

    display(artist_box, fetch_artist_button, back_button)

    def fetch_artist(btn):
        artist = artist_box.value
        db = db_widget.value

        if not artist:
            clear_output()
            lbl.value = "String cannot be empty."
            display(lbl, back_button)
            return
        try:
            df = art.fetch_artist_data(db, artist)

            if df is None:
                clear_output()
                lbl.value = f"'{artist}' cannot be found in the database."
                display(lbl, back_button)
                return

            if df.empty:
                clear_output()
                lbl.value = f"No data available for '{artist}'."
                display(lbl, back_button)
                return

            clear_output()
            lbl.value = f"{artist.title()} found in database."
            table_widget = widgets.Button(
                description="View Table", style=style)
            graph_widget = widgets.Button(
                description="View Graph", style=style)

            display(lbl, table_widget, graph_widget, back_button)

            def table_wrap(btn):
                fetch_table(art.format_table, df, artist)
                display(graph_widget, back_button)

            def graph_wrap(btn):
                fetch_graph(art.format_graph, df, artist)
                display(table_widget, back_button)

        except Exception as e:
            lbl.value = f"Error: {e}"
            display(lbl, back_button)

        table_widget.on_click(table_wrap)
        graph_widget.on_click(graph_wrap)
        back_button.on_click(go_to_main)

    fetch_artist_button.on_click(fetch_artist)
    back_button.on_click(go_to_main)


def clicked_top5(btn):
    clear_output()
    range_slider = widgets.IntRangeSlider(
        value=(1998, 2020), min=1998, max=2020)
    fetch_year_button = widgets.Button(
        description="Fetch Years Data", style=style)

    display(range_slider, fetch_year_button, back_button)

    def fetch_data(btn):
        db = db_widget.value
        start_year = range_slider.value[0]
        end_year = range_slider.value[1]

        try:
            df = top.fetch_and_process_data(db, start_year, end_year)
            if df.empty:
                clear_output()
                lbl.value = f"No data available for \
                                    {start_year}-{end_year}."
                display(lbl, back_button)
                return
            top5 = top.top5_prep(df, start_year, end_year)

            clear_output()
            lbl.value = f"Top 5 artists from {start_year}-{end_year}"
            table_widget = widgets.Button(
                description="View Table", style=style)
            graph_widget = widgets.Button(
                description="View Graph", style=style)

            display(lbl, table_widget, graph_widget, back_button)

            def table_wrap(btn):
                fetch_table(top.format_table, top5, start_year, end_year)
                display(graph_widget, back_button)

            def graph_wrap(btn):
                fetch_graph(top.format_graph, top5, start_year, end_year)
                display(table_widget, back_button)

        except Exception as e:
            lbl.value = f"Error: {e}"
            display(lbl, back_button)

        table_widget.on_click(table_wrap)
        graph_widget.on_click(graph_wrap)
        back_button.on_click(go_to_main)

    fetch_year_button.on_click(fetch_data)
    back_button.on_click(go_to_main)


db_check_widget.on_click(clicked_check_db)
genre_widget.on_click(clicked_genre)
artist_widget.on_click(clicked_artist)
top5_widget.on_click(clicked_top5)

display(db_widget, db_check_widget, lbl)

Text(value='CWDatabase.db', description='Database: ', placeholder='Enter database filename')

Button(description='Check Database', style=ButtonStyle())